## Importar librerías

In [ ]:
import os
import io
import pandas as pd
import matplotlib.pyplot as plt
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import configparser

## Configuración de correo

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

SMTP_SERVER = config['EMAIL']['SMTP_SERVER']
SMTP_PORT = config['EMAIL']['SMTP_PORT']
EMAIL_SENDER = config['EMAIL']['EMAIL_SENDER']
EMAIL_PASSWORD = config['EMAIL']['EMAIL_PASSWORD']
EMAIL_RECEIVER = config['EMAIL']['EMAIL_RECEIVER']

## Funciones de filtrado

In [ ]:
def contiene_grado_y_estudiante(text):
    if not isinstance(text, str):
        return False
    texto_lower = text.lower()
    return 'grado' in texto_lower and 'estudiante' in texto_lower

def fila_a_eliminar(row):
    textos = row.astype(str)
    contiene_grado = textos.str.contains('grado', case=False, na=False).any()
    contiene_estudiante = textos.str.contains('estudiante', case=False, na=False).any()
    return contiene_grado and contiene_estudiante

def cargar_y_filtrar_excel(carpeta):
    dataframes = []
    for archivo in os.listdir(carpeta):
        if archivo.endswith('.xlsx'):
            ruta_archivo = os.path.join(carpeta, archivo)
            df = pd.read_excel(ruta_archivo)
            print(f"Nombres de columnas en archivo {archivo}: {df.columns.tolist()}")
            df_filtrado = df[~df.apply(fila_a_eliminar, axis=1)]
            df_filtrado = df_filtrado.dropna(how='all')
            df_filtrado = df_filtrado.dropna(axis=1, how='all')
            dataframes.append(df_filtrado)
    if dataframes:
        df_combinado = pd.concat(dataframes, ignore_index=True)
    else:
        df_combinado = pd.DataFrame()
    return df_combinado

## Funciones de conteo

In [ ]:
def contar_ausencias_por_dia_y_grado(df):
    columnas_dias = [col for col in df.columns if str(col).strip().isdigit()]
    if 'GRADO' not in df.columns:
        raise KeyError('La columna "GRADO" no se encuentra en el DataFrame.')
    df_ausencias = df.groupby('GRADO')[columnas_dias].apply(lambda x: x.notna().sum())
    df_ausencias = df_ausencias.reset_index()
    return df_ausencias

def contar_asistencias_por_dia_y_grado(df):
    columnas_dias = [col for col in df.columns if str(col).strip().isdigit()]
    if 'GRADO' not in df.columns:
        raise KeyError('La columna "GRADO" no se encuentra en el DataFrame.')
    df_ausencias = df.groupby('GRADO')[columnas_dias].apply(lambda x: x.notna().sum())
    df_asistencias = df.groupby('GRADO')[columnas_dias].apply(lambda x: x.isna().sum())
    df_resultado = pd.concat([
        df_ausencias.rename(columns=lambda x: f'Ausencias_{x}'),
        df_asistencias.rename(columns=lambda x: f'Asistencias_{x}')
    ], axis=1)
    df_resultado = df_resultado.reset_index()
    return df_resultado

## Funciones de gráficos

In [ ]:
def etiqueta_con_cantidad(pct, allvals):
    absolute = int(round(pct / 100. * sum(allvals)))
    return f"{absolute} ({pct:.1f}%)"

## Función para enviar correo

In [ ]:
def enviar_email_con_graficos(asunto, cuerpo_texto, figuras, 
                              server=SMTP_SERVER, port=SMTP_PORT,
                              sender=EMAIL_SENDER, password=EMAIL_PASSWORD, receiver=EMAIL_RECEIVER):
    """
    Envía un correo con gráficos embebidos.
    - figuras: lista de tuplas (matplotlib.figure.Figure, id_cid_str)
    """
    msg = MIMEMultipart('related')
    msg['Subject'] = asunto
    msg['From'] = sender
    msg['To'] = receiver

    # Crear cuerpo HTML con referencias a imágenes cid
    html_body = f"<html><body><p>{cuerpo_texto}</p>"
    for _, cid in figuras:
        html_body += f'<img src="cid:{cid}" style="max-width:100%; height:auto;"><br>'
    html_body += "</body></html>"

    msg_alternative = MIMEMultipart('alternative')
    msg.attach(msg_alternative)

    msg_text = MIMEText(html_body, 'html')
    msg_alternative.attach(msg_text)

    # Adjuntar imágenes de cada figura como MIMEImage
    for fig, cid in figuras:
        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        img = MIMEImage(buf.read())
        img.add_header('Content-ID', f'<{cid}>')
        img.add_header('Content-Disposition', 'inline', filename=f'{cid}.png')
        msg.attach(img)
        buf.close()

    # Enviar el correo
    try:
        with smtplib.SMTP(server, port) as smtp:
            smtp.starttls()
            smtp.login(sender, password)
            smtp.sendmail(sender, receiver, msg.as_string())
        print('Correo enviado correctamente.')
    except Exception as e:
        print('Error enviando el correo:', str(e))

## Función principal

In [ ]:
def main():
    carpeta = 'files'
    df_resultado = cargar_y_filtrar_excel(carpeta)
    print(f'Todos los archivos Excel en "{carpeta}" han sido combinados, filtrados y ordenados.')

    if df_resultado.empty:
        print("No hay datos para procesar.")
        return
    
    df_resultado['GRADO'] = df_resultado['GRADO'].astype(str).str.replace(' ', '')
    df_resultado_final = contar_asistencias_por_dia_y_grado(df_resultado)
    print("Conteo de ausencias y asistencias por día y grado:")
    print(df_resultado_final.head(30))

    ultima_columna_ausencias = df_resultado_final.filter(like='Ausencias_').columns[-1]
    ultima_columna_asistencias = df_resultado_final.filter(like='Asistencias_').columns[-1]

    total_ausencias_ultimo_dia = df_resultado_final[ultima_columna_ausencias].sum()
    total_asistencias_ultimo_dia = df_resultado_final[ultima_columna_asistencias].sum()

    print(f'Total de Ausencias del Último Día: {total_ausencias_ultimo_dia}')
    print(f'Total de Asistencias del Último Día: {total_asistencias_ultimo_dia}')

    # Gráfico circular
    cantidades = [total_asistencias_ultimo_dia, total_ausencias_ultimo_dia]
    categorias = ['Asistencias', 'Ausencias']
    fig_circular, ax_circular = plt.subplots(figsize=(8, 6))
    ax_circular.pie(cantidades, labels=categorias, autopct=lambda pct: etiqueta_con_cantidad(pct, cantidades),
                    startangle=90, colors=['#4CAF50', '#F44336'])
    ax_circular.set_title('Proporción y Cantidad de Asistencias y Ausencias del Último Día', fontsize=14, fontweight='bold')
    ax_circular.axis('equal')

    # Totalizar asistencias y ausencias por grado
    total_ausencias = df_resultado_final.filter(like='Ausencias_').sum(axis=1)
    total_asistencias = df_resultado_final.filter(like='Asistencias_').sum(axis=1)
    df_resultado_final['Total_Ausencias'] = total_ausencias
    df_resultado_final['Total_Asistencias'] = total_asistencias
    df_ausencias = df_resultado_final[['GRADO', 'Total_Ausencias', 'Total_Asistencias']]
    print("Total de ausencias y asistencias por grado:")
    print(df_ausencias.head(30))

    # Gráfico barras
    total_ausencias_por_grado = df_resultado_final.groupby('GRADO')['Total_Ausencias'].sum()
    fig_barras, ax_barras = plt.subplots(figsize=(10, 6))
    total_ausencias_por_grado.plot(kind='bar', color='skyblue', ax=ax_barras)
    ax_barras.set_xlabel('Grado', fontsize=12)
    ax_barras.set_ylabel('Total de Ausencias', fontsize=12)
    ax_barras.set_title('Total de Ausencias por Grado en el Mes', fontsize=14, fontweight='bold')
    ax_barras.tick_params(axis='x', rotation=45)
    ax_barras.grid(axis='y')
    plt.tight_layout()

    plt.close(fig_circular)
    plt.close(fig_barras)

    # Enviar email con gráficos
    asunto_email = 'Informe Gráficos de Asistencias y Ausencias'
    cuerpo_email = 'A continuación se incluyen los gráficos de asistencias y ausencias generados automáticamente.'
    figuras = [
        (fig_circular, 'grafico_circular'),
        (fig_barras, 'grafico_barras')
    ]
    enviar_email_con_graficos(asunto_email, cuerpo_email, figuras)

if __name__ == '__main__':
    main()
